# Check GPU

It is recommended to run this notebook with GPU support. If you have an Nvidea graphics card and drivers installed, the following block of code should show the details of the installed GPU.

In [ ]:
!nvidia-smi

# Untar Testing Data

In the code block below, specify the path to the saved testing data in tar format. This will untar the data into a folder of the same name in the parent directory of this notebook.

In [ ]:
!tar -xkf '/path/to/saved/tar/file' -C '/path/to/save/untarred/files'

# Import Packages

In [ ]:
import torch
import numpy as np

import os
import glob
import random
import pickle

In [ ]:
# Import model architectures and data structures

os.chdir('path/to/repository')
from test_utils import get_model_error

# Set Hyperparameters

In [ ]:
# Set random seed, define device

seed = 3
torch.manual_seed(seed)
torch.use_deterministic_algorithms(True)
np.random.seed(seed)
random.seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Set batch size
test_batch_size = 1024

# Manually set values for buildings, unsampled locations, and sampled locations in the environment mask. 
# For the models in the PIMRC paper, these are set to "None", meaning they keep the default values of -1, 0, and 1 respectively.
building_value = None
unsampled_value = None
sampled_value = None

Set the folder to load trained models from. All models in the selected model_folder will be tested. Additionally create and specify a folder to save the results to, and specify the paths to the saved data and data scaler (located within this repository).

In [ ]:
# Set folder to load model(s) to test
model_folder = '/folder/with/trained/models'

# Set folder of results to save current results
results_folder = '/folder/to/save/results'

# Specify path to test data
test_data_folder = '/path/to/saved/testing/data'

# Specify path to data scaler
scaler_path = 'scalers/minmax_scaler_zero_min134.joblib'

In [ ]:
model_names = glob.glob(os.path.join(model_folder, '*.pth'))
for model_path in model_names:
  error = get_model_error(model_path, scaler_path, building_value=building_value, sampled_value=sampled_value)
  filename = os.path.basename(model_path).split('.')[0] + '.pickle'
  with open(os.path.join(results_folder, filename), 'wb') as f:
    pickle.dump(error, f)